In [ ]:
from tensorflow import keras
fmnist = keras.datasets.fashion_mnist

(X_train, y_train), (X_test, y_test) = fmnist.load_data()
X_train, X_test = X_train / 255.0 , X_test / 255.0
from tensorflow.keras import layers
import pandas as pd
import matplotlib.pyplot as plt
import os
import pickle
root_logdir = os.path.join(os.curdir, "my_tb_logs")

In [ ]:
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d_%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()

In [ ]:
def MLP():
    model = keras.models.Sequential()
    model.add(layers.Flatten(input_shape=[28, 28]))
    model.add(layers.Dense(300, activation="relu"))
    model.add(layers.Dense(100, activation="relu"))
    model.add(layers.Dense(10, activation="softmax"))
    return model

In [ ]:
def compiler(lr=0, model=MLP(), op="sgd"):
    if op=="sgd":
        optimizer = keras.optimizers.SGD(learning_rate=lr) if lr!=0 else keras.optimizers.SGD()
    elif op=="nes":
        optimizer = keras.optimizers.SGD(learning_rate=lr, momentum=0.9, nesterov=True) if lr!=0 else keras.optimizers.SGD(momentum=0.9, nesterov=True)
    elif op=="adg":
        optimizer = keras.optimizers.Adagrad(learning_rate=lr) if lr!=0 else keras.optimizers.Adagrad()
    elif op=="rms":
        optimizer = keras.optimizers.RMSprop(learning_rate=lr) if lr!=0 else keras.optimizers.RMSprop()
    elif op=="adm":
        optimizer = keras.optimizers.Adam(learning_rate=lr) if lr!=0 else keras.optimizers.Adam()
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

In [ ]:
def trainer(lr, model):
    name = "model_0515_(lr)" + str(lr) + ".h5"
    cp = keras.callbacks.ModelCheckpoint(name, save_best_only=True)
    es = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
    tb = keras.callbacks.TensorBoard(run_logdir)
    history = model.fit(X_train, y_train, epochs=10, validation_split=0.2, callbacks=[cp, es, tb])
    hisc = "history_(lr)" +  str(lr) + ".csv"
    hd = pd.DataFrame(history.history)
    with open(hisc, mode='w') as f:
        hd.to_csv(f)
    return history

In [ ]:
def grapher(lr):
    name = "history_(lr)" + str(lr) + ".csv"
    df = pd.read_csv(name, index_col=0)
    df.plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0, 1)
    plt.show()

In [ ]:
def test(lr):
    name = "model_0515_(lr)" + str(lr) + ".h5"
    model = keras.models.load_model(name)
    test = model.evaluate(X_test, y_test)
    return test

In [ ]:
model = MLP()

In [ ]:
model2 = compiler(lr=0.01, model=model)

In [ ]:
model2.summary()

In [ ]:
trainer(0.01, model2)

In [ ]:
grapher(0.01)

In [ ]:
test(0.01)